In [1]:
import ssl
from random import randint
from time import sleep

from bs4 import BeautifulSoup
import urllib
import csv
import json
from pprint import pprint

import pandas as pd
import numpy as np

In [6]:
datapath_cgu = 'C:/Users/LauraSantAnnaGualdaP/Desktop/Dissertacao/DATA/data_cgu/'
file_cgu = 'CGU_total_irreg_dados.xlsx'
data_cgu = pd.read_excel(datapath_cgu+file_cgu)
list_mun_audit = list(data_cgu['Cod'].unique())
len(list_mun_audit)

file_errados = 'lista_errados.xlsx'
errados = pd.read_excel(file_errados)
list_errados = list(errados['cod_certo'])

In [7]:
ssl._create_default_https_context = ssl._create_unverified_context


# codigos de municipios e codigos de estados


municipios = json.load(open('municipios.json', encoding="utf8"))

# paginacao code list
pages = [2008, 2013]

In [8]:
# url former
dic_mun_audit = {}
lista = []
for municipio in municipios:
    dic_temp = {}
    if (municipio['codigo_ibge'] in list_errados) == True:
        codigo_uf = str(municipio['codigo_uf'])
        codigo_mun = str(municipio['codigo_ibge'])[:-1] # fix for codigo mun on url
        dic_temp['nome_mun'] = municipio['nome_municipio']
        dic_temp['codigo_mun'] = codigo_mun # without the last number
        dic_temp['nome_uf'] =  str(municipio['uf'])
        dic_temp['codigo_uf'] = codigo_uf
        
        dic_temp[2008] = 'https://www.fnde.gov.br/siope/indicadoresFinanceirosEEducacionais.do?acao=PESQUISAR&anoPaginacao=' + '2008' + '&paginacao=-&pag=result&cod_uf=' + codigo_uf + '&municipios=' + codigo_mun
        dic_temp[2013] = 'https://www.fnde.gov.br/siope/indicadoresFinanceirosEEducacionais.do?acao=PESQUISAR&anoPaginacao=' + '2013' + '&paginacao=%2B&pag=result&cod_uf=' + codigo_uf + '&municipios=' + codigo_mun
        #for page in pages:
        #        codigo_page = str(page)
        #        url = 'https://www.fnde.gov.br/siope/indicadoresFinanceirosEEducacionais.do?acao=PESQUISAR&anoPaginacao=' + codigo_page + '&paginacao=-' + '&pag=result&cod_uf=' + codigo_uf + '&municipios=' + codigo_mun
        #        dic_temp[page] = url
        dic_mun_audit[municipio['codigo_ibge']] = dic_temp
        lista.append(municipio['codigo_ibge'])
        
print(lista)

[4103206, 1301100, 3122009, 5213756, 3145604, 3148103, 4121307, 3545209, 2612455, 2110005, 2412500, 3556354, 4323507]


In [9]:
dic_mun_audit

{1301100: {'nome_mun': 'Careiro',
  'codigo_mun': '130110',
  'nome_uf': 'AM',
  'codigo_uf': '13',
  2008: 'https://www.fnde.gov.br/siope/indicadoresFinanceirosEEducacionais.do?acao=PESQUISAR&anoPaginacao=2008&paginacao=-&pag=result&cod_uf=13&municipios=130110',
  2013: 'https://www.fnde.gov.br/siope/indicadoresFinanceirosEEducacionais.do?acao=PESQUISAR&anoPaginacao=2013&paginacao=%2B&pag=result&cod_uf=13&municipios=130110'},
 2110005: {'nome_mun': 'Santa Luzia',
  'codigo_mun': '211000',
  'nome_uf': 'MA',
  'codigo_uf': '21',
  2008: 'https://www.fnde.gov.br/siope/indicadoresFinanceirosEEducacionais.do?acao=PESQUISAR&anoPaginacao=2008&paginacao=-&pag=result&cod_uf=21&municipios=211000',
  2013: 'https://www.fnde.gov.br/siope/indicadoresFinanceirosEEducacionais.do?acao=PESQUISAR&anoPaginacao=2013&paginacao=%2B&pag=result&cod_uf=21&municipios=211000'},
 2412500: {'nome_mun': 'São Miguel',
  'codigo_mun': '241250',
  'nome_uf': 'RN',
  'codigo_uf': '24',
  2008: 'https://www.fnde.gov.b

In [32]:
errors = 0
lastMun = 0
while errors < 10:
    def crawler_siope(list_mun_audit, init):
        print('starting function on', init)
        list_mun = list_mun_audit[init:]
        for i in list_mun:
            lastMun = i
            municipio = dic_mun_audit[i]
            for page in pages:
                #print('url:', municipio[page])
                html = urllib.request.urlopen(municipio[page]).read()
                soup = BeautifulSoup(html, 'html.parser')
                tableList = soup.select(".table")
                if (len(tableList) > 3):
                    table = tableList[3] #tabela 4 -> é a de despesa por aluno

                    yearLines = table.select(".rowA")
                    yearLines.extend(table.select(".rowB"))
                    # parser
                    for col in yearLines:
                    #col = yearLines[0]
                        dic_gasto = {}
                        for yearValue in col.select("td"):
                            if len(yearValue.select('.text')) > 0:
                                gasto = str(yearValue.select('.text')[0].text).strip()
                            if len(yearValue.select('a')) > 0:
                                index = str(yearValue.select('a')[0]).find('anos')
                                ano = str(yearValue.select('a')[0])[index+5:index+9]
                                if len(yearValue.select('.number')) > 0:
                                    valor = str(yearValue.select('.number')[0].text).strip()
                                    dic_gasto[ano] = valor.replace(u'\xa0', u'')

                        municipio[gasto+"_"+str(page)] = dic_gasto
                    print(municipio['nome_mun'], page, ' OK') 
                else:
                    print(municipio['nome_mun'] + ' ERRO')
        return i
    try:
        i = crawler_siope(list_errados, lastMun)
        print(i)
    except:
        print('error')
        lastMun = list_errados.index(i)
        print(lastMun)
        errors = errors + 1   

starting function on 0
Careiro ERRO
Careiro 2013  OK
Montividiu 2008  OK
Montividiu 2013  OK
Santa Luzia 2008  OK
Santa Luzia ERRO
Patrocínio 2008  OK
Patrocínio 2013  OK
Divino 2008  OK
Divino ERRO
Oliveira 2008  OK
Oliveira 2013  OK
Santa Cruz ERRO
Santa Cruz 2013  OK
Bom Sucesso 2008  OK
Bom Sucesso 2013  OK
Rancho Alegre 2008  OK
Rancho Alegre 2013  OK
São Miguel 2008  OK
São Miguel 2013  OK
Vista Alegre 2008  OK
error
11
starting function on 11
Salto ERRO
Salto 2013  OK
Vargem 2008  OK
Vargem ERRO
3556354
starting function on 11
Salto 2008  OK
Salto 2013  OK
Vargem 2008  OK
Vargem 2013  OK
3556354
starting function on 11
Salto 2008  OK
error
12
starting function on 12
Vargem 2008  OK
Vargem ERRO
3556354
starting function on 12
Vargem 2008  OK
Vargem 2013  OK
3556354
starting function on 12
Vargem 2008  OK
Vargem 2013  OK
3556354
starting function on 12
Vargem 2008  OK
Vargem ERRO
3556354
starting function on 12
Vargem 2008  OK
Vargem 2013  OK
3556354
starting function on 12
Vargem

In [33]:
dic_mun_audit[3545209]

{'nome_mun': 'Salto',
 'codigo_mun': '354520',
 'nome_uf': 'SP',
 'codigo_uf': '35',
 2008: 'https://www.fnde.gov.br/siope/indicadoresFinanceirosEEducacionais.do?acao=PESQUISAR&anoPaginacao=2008&paginacao=-&pag=result&cod_uf=35&municipios=354520',
 2013: 'https://www.fnde.gov.br/siope/indicadoresFinanceirosEEducacionais.do?acao=PESQUISAR&anoPaginacao=2013&paginacao=%2B&pag=result&cod_uf=35&municipios=354520',
 'Investimento educacional por aluno do ensino fundamental_2008': {'2008': 'R$5.112,81',
  '2009': 'R$6.633,14',
  '2010': 'R$8.129,84',
  '2011': 'R$9.467,83',
  '2012': 'R$7.538,13',
  '2013': 'R$9.429,39'},
 'Investimento educacional por aluno da educação superior_2008': {'2008': 'R$0,00',
  '2009': 'R$0,00',
  '2010': 'R$0,00',
  '2011': 'R$0,00',
  '2012': 'R$0,00',
  '2013': 'R$0,00'},
 'Investimento educacional por aluno da educação especial_2008': {'2008': 'R$0,00',
  '2009': 'R$0,00',
  '2010': 'R$0,00',
  '2011': 'R$0,00',
  '2012': 'R$0,00',
  '2013': 'R$0,00'},
 'Inves

In [34]:
dic_var = {}
for i in list_errados:
    print(i)
    x = dic_mun_audit[i]['Investimento educacional por aluno do ensino fundamental_2008']
    y = dic_mun_audit[i]['Investimento educacional por aluno do ensino fundamental_2013']
    #nome_mun = dic_mun_audit[i]['nome_mun']
    #nome_uf = dic_mun_audit[i]['nome_uf']
    dic_var[i] = {**x, **y}
invest_aluno_ef = pd.DataFrame.from_dict(dic_var, orient='index')

1301100
5213756
2110005
3148103
3122009
3145604
2612455
4103206
4121307
2412500
4323507
3545209
3556354


In [35]:
invest_aluno_ef.to_excel('Invest_aluno_EF_errados.xlsx', encoding='utf8')

In [36]:
all_ind = pd.DataFrame.from_dict(dic_mun_audit, orient='index').reset_index()
all_ind

,index,nome_mun,codigo_mun,nome_uf,codigo_uf,2008,2013,Investimento educacional por aluno do ensino fundamental_2008,Investimento educacional por aluno da educação superior_2008,Investimento educacional por aluno da educação especial_2008,...,Investimento educacional por aluno da educação de jovens e adultos_2013,Investimento educacional por aluno da educação profissional_2013,Investimento educacional por aluno_2013,Despesas com profissionais não docentes da área educacional por aluno da educação básica_2013,Investimento educacional por aluno do ensino fundamental_2013,Investimento educacional por aluno da educação superior_2013,Investimento educacional por aluno da educação especial_2013,Investimento educacional por aluno da educação básica_2013,Despesa com professores por aluno da educação básica_2013,Percentual de investimento por aluno da educação superior em relação ao investimento por aluno da educação básica_2013
0,1301100,Careiro,130110,AM,13,https://www.fnde.gov.br/siope/indicadoresFinan...,https://www.fnde.gov.br/siope/indicadoresFinan...,"{'2008': 'R$1.974,69', '2009': 'R$2.084,61', '...","{'2008': 'R$0,00', '2009': 'R$0,00', '2010': '...","{'2008': 'R$34,78', '2009': 'R$0,00', '2010': ...",...,"{'2014': 'R$1.639,34', '2015': 'R$1.412,42', '...","{'2014': 'R$0,00', '2015': 'R$0,00', '2016': '...","{'2014': 'R$4.048,38', '2015': 'R$3.484,48', '...","{'2014': 'R$459,72', '2015': 'R$439,96', '2016...","{'2014': 'R$4.325,18', '2015': 'R$3.848,66', '...","{'2014': 'R$0,00', '2015': 'R$0,00', '2016': '...","{'2014': 'R$2.173,91', '2015': 'R$2.830,18', '...","{'2014': 'R$4.048,38', '2015': 'R$3.484,48', '...","{'2014': 'R$1.887,06', '2015': 'R$1.959,81', '...","{'2014': '0,00%', '2015': '0,00%', '2016': '0,..."
1,2110005,Santa Luzia,211000,MA,21,https://www.fnde.gov.br/siope/indicadoresFinan...,https://www.fnde.gov.br/siope/indicadoresFinan...,"{'2008': 'R$1.431,94', '2009': 'R$1.656,20', '...","{'2008': 'R$0,00', '2009': 'R$0,00', '2010': '...","{'2008': 'R$0,00', '2009': 'R$0,00', '2010': '...",...,"{'2014': 'R$1.846,15', '2015': 'R$1.416,87', '...","{'2014': 'R$0,00', '2015': 'R$0,00', '2016': '...","{'2014': 'R$2.995,65', '2015': 'R$3.423,18', '...","{'2014': 'R$6,78', '2015': 'R$61,41', '2016': ...","{'2014': 'R$3.257,05', '2015': 'R$3.929,60', '...","{'2014': 'R$0,00', '2015': 'R$0,00', '2016': '...","{'2014': 'R$10.000,00', '2015': 'R$0,00', '201...","{'2014': 'R$2.995,65', '2015': 'R$3.423,18', '...","{'2014': 'R$1.696,75', '2015': 'R$2.916,45', '...","{'2014': '0,00%', '2015': '0,00%', '2016': '0,..."
2,2412500,São Miguel,241250,RN,24,https://www.fnde.gov.br/siope/indicadoresFinan...,https://www.fnde.gov.br/siope/indicadoresFinan...,"{'2008': 'R$2.198,01', '2009': 'R$2.524,31', '...","{'2008': 'R$0,00', '2009': 'R$0,00', '2010': '...","{'2008': 'R$0,00', '2009': 'R$0,00', '2010': '...",...,"{'2014': 'R$0,00', '2015': 'R$0,00', '2016': '...","{'2014': 'R$0,00', '2015': 'R$0,00', '2016': '...","{'2014': 'R$3.456,66', '2015': 'R$4.026,37', '...","{'2014': 'R$359,46', '2015': 'R$370,86', '2016...","{'2014': 'R$3.816,51', '2015': 'R$4.406,59', '...","{'2014': 'R$0,00', '2015': 'R$0,00', '2016': '...","{'2014': 'R$2.185,90', '2015': 'R$0,00', '2016...","{'2014': 'R$3.456,66', '2015': 'R$4.026,37', '...","{'2014': 'R$2.392,98', '2015': 'R$2.926,58', '...","{'2014': '0,00%', '2015': '0,00%', '2016': '0,..."
3,2612455,Santa Cruz,261245,PE,26,https://www.fnde.gov.br/siope/indicadoresFinan...,https://www.fnde.gov.br/siope/indicadoresFinan...,"{'2008': 'R$1.774,37', '2009': 'R$2.371,57', '...","{'2008': 'R$0,00', '2009': 'R$0,00', '2010': '...","{'2008': 'R$0,00', '2009': 'R$0,00', '2010': '...",...,"{'2014': 'R$2.385,59', '2015': 'R$1.604,70', '...","{'2014': 'R$0,00', '2015': 'R$0,00', '2016': '...","{'2014': 'R$3.568,95', '2015': 'R$3.715,71', '...","{'2014': 'R$0,00', '2015': 'R$717,36', '2016':...","{'2014': 'R$3.768,02', '2015': 'R$4.313,39', '...","{'2014': 'R$0,00', '2015': 'R$0,00', '2016': '...","{'2014': 'R$0,00', 

In [37]:
#all_ind = invest_aluno_ef.drop([invest_aluno_ef.columns[2], invest_aluno_ef.columns[5], invest_aluno_ef.columns[6]], axis=1)
all_ind.to_excel('SIOPE_ind_raw_errados.xlsx', encoding='utf8')

In [39]:
dic_mun_audit[5003504]

{'nome_mun': 'Douradina',
 'codigo_mun': '500350',
 'nome_uf': 'MS',
 'codigo_uf': '50',
 2008: 'https://www.fnde.gov.br/siope/indicadoresFinanceirosEEducacionais.do?acao=PESQUISAR&anoPaginacao=2008&paginacao=-&pag=result&cod_uf=50&municipios=500350',
 2013: 'https://www.fnde.gov.br/siope/indicadoresFinanceirosEEducacionais.do?acao=PESQUISAR&anoPaginacao=2013&paginacao=%2B&pag=result&cod_uf=50&municipios=500350'}